In [119]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
import pandas as pd
import re
import requests
import seaborn as sns
import time
import descartes
import geopandas as gpd

from arcgis.geocoding import geocode
from arcgis.gis import GIS
from dotenv import load_dotenv
from functools import reduce
from shapely.geometry import Point, Polygon
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder, RobustScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeRegressor
from textwrap import wrap
from xgboost import XGBClassifier, XGBRegressor, plot_importance
from yellowbrick.classifier import ROCAUC, ConfusionMatrix

plt.style.use("ggplot")
sns.set_context("talk")

In [2]:
load_dotenv("")
censuskey = os.getenv("censuskey")

In [207]:
var_description = pd.DataFrame(pd.DataFrame(
    requests.get("https://api.census.gov/data/2016/acs/acs5/profile/variables.json")
                                            .json()["variables"])
                               .T.iloc[:, 3]
                               .str.title()
                               .str.split("!!", expand=True)
                              ).sort_index()

In [208]:
var_description.columns=["unit", 
                         "subject",
                         "denominator",
                         "criteria_1",
                         "criteria_2",
                         "criteria_3",
                         "criteria_4"
                        ]

In [213]:
var_description

,unit,subject,denominator,criteria_1,criteria_2,criteria_3,criteria_4
AIANHH,American Indian Area/Alaska Native Area/Hawaii...,None,None,None,None,None,None
AIHHTLI,American Indian Area (Off-Reservation Trust La...,None,None,None,None,None,None
AITSCE,American Indian Tribal Subdivision (Census),None,None,None,None,None,None
ANRC,Alaska Native Regional Corporation,None,None,None,None,None,None
CBSA,Metropolitan Statistical Area/Micropolitan Sta...,None,None,None,None,None,None
CD115,Congressional District (115Th),None,None,None,None,None,None
CNECTA,Combined New England City And Town Area,None,None,None,None,None,None
CONCIT,Consolidated City,None,None,None,None,None,None
COUNTY,County (Fips),None,None,None,None,None,None
COUSUB,County Subdivision (Fips),None,None,None,None,None,None


In [210]:
var_description.to_csv("../data/var_description.csv")

In [211]:
variables_selected = var_description.loc[((~var_description.index.str.contains(r'DP')|
                                           (var_description.index.str.contains(r'DP')&
                                            var_description.index.str.contains(r'[0-9]PE')))&
                                          ~var_description.index.str.contains(r'PR_'))
                                        ].sort_index()

In [109]:
len(variables_selected)

554

In [193]:
varlist_dict = {}
var_dict = {}
for year in range(2010, 2018):
    varlist_dict[year] = []
    k = 0
    for var in variables_selected.index:
        success = False
        l = 0
        while not success:
            l += 1
            resp = requests.get(
                "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
                .format(year, var, censuskey))
            time.sleep(1)
            if resp.status_code ==200:
                varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
                success = True
            elif l > 10:
                success = True
        k += 1
        
        if k%int(len(variables_selected)/10)==0:
            print("---progress: {:0.2f}% completed for {}---".format(
                k*10/int(len(variables_selected)/10), 
                year
            ))
    var_dict[year] = reduce(lambda left, right: pd.merge(left, 
                                                         right, 
                                                         how='outer'
                                                        ), varlist_dict[year])
    print(year, var_dict[year].shape)

---progress: 10.00% completed for 2010---
---progress: 20.00% completed for 2010---
---progress: 30.00% completed for 2010---
---progress: 40.00% completed for 2010---
---progress: 50.00% completed for 2010---
---progress: 60.00% completed for 2010---
---progress: 70.00% completed for 2010---
---progress: 80.00% completed for 2010---
---progress: 90.00% completed for 2010---
---progress: 100.00% completed for 2010---
2010 (3221, 544)
---progress: 10.00% completed for 2011---
---progress: 20.00% completed for 2011---
---progress: 30.00% completed for 2011---
---progress: 40.00% completed for 2011---
---progress: 50.00% completed for 2011---
---progress: 60.00% completed for 2011---
---progress: 70.00% completed for 2011---
---progress: 80.00% completed for 2011---
---progress: 90.00% completed for 2011---
---progress: 100.00% completed for 2011---
2011 (3221, 545)
---progress: 10.00% completed for 2012---
---progress: 20.00% completed for 2012---
---progress: 30.00% completed for 2012--

In [459]:
missing_varlist_dict = {}
missing_var_dict = {}
for year in range(2010, 2018):
    missing_col = []
    for col in var_dict[2016].columns:
        if col not in var_dict[2010].columns:
            missing_col.append(col)
    missing_varlist_dict[year] = []
    k = 0
    for var in missing_col:
        success = False
        l = 0
        while not success:
            l += 1
            resp = requests.get(
                "https://api.census.gov/data/{}/acs/acs5/profile?get={}&for=county:*&in=state:*&key={}"
                .format(year, var, censuskey))
#             time.sleep(1)
            if resp.status_code ==200:
                missing_varlist_dict[year].append(pd.DataFrame(resp.json()[1:], columns=resp.json()[0]))
                success = True
            elif l > 10:
                success = True
                print("no data for: ", year, var)
        k += 1

no data for:  2010 CD115
no data for:  2010 DP02_0150PE
no data for:  2010 DP02_0151PE
no data for:  2010 DP02_0152PE
no data for:  2010 DP04_0142PE
no data for:  2010 DP04_0143PE
no data for:  2010 DP05_0082PE
no data for:  2010 DP05_0083PE
no data for:  2010 DP05_0084PE
no data for:  2010 ZCTA5
no data for:  2011 CD115
no data for:  2011 DP02_0150PE
no data for:  2011 DP02_0151PE
no data for:  2011 DP02_0152PE
no data for:  2011 DP04_0142PE
no data for:  2011 DP04_0143PE
no data for:  2011 DP05_0082PE
no data for:  2011 DP05_0083PE
no data for:  2011 DP05_0084PE
no data for:  2012 CD115
no data for:  2012 DP02_0150PE
no data for:  2012 DP02_0151PE
no data for:  2012 DP02_0152PE
no data for:  2012 DP04_0142PE
no data for:  2012 DP04_0143PE
no data for:  2012 DP05_0082PE
no data for:  2012 DP05_0083PE
no data for:  2012 DP05_0084PE
no data for:  2013 CD115
no data for:  2013 DP04_0142PE
no data for:  2013 DP04_0143PE
no data for:  2013 DP05_0082PE
no data for:  2013 DP05_0083PE
no data

In [467]:
for year in range(2010, 2018):
    var_dict[year] = var_dict[year].apply(pd.to_numeric, errors='coerce')
    var_dict[year] = var_dict[year].loc[var_dict[year].state!=72]
    var_dict[year]["FIPS"] = var_dict[year].state * 1000 + var_dict[year].county
    var_dict[year].to_csv("../data/census_{}.csv".format(year))

In [265]:
pd.options.display.float_format = '{:,.2f}'.format

In [461]:
for year in range(2010, 2018):
    var_dict[year] = var_dict[year].applymap(
        lambda x: np.nan if x==-666666666 or x==-888888888 or x==-999999999 else x
    )

In [478]:
for year in range(2010, 2018):
    df = var_dict[year].drop(
        columns=var_dict[year].loc[
            0, 
            ((var_dict[year].isnull().sum()>0)&(var_dict[year].isnull().sum()!=var_dict[year].shape[0]))
        ].index
    )
    df = df.loc[:, df.columns.str.contains(r"DP0")].dropna(axis=1)
    df_target = var_dict[year].loc[
        :, var_dict[year].loc[
            0,
            ((var_dict[year].isnull().sum()>0)&(var_dict[year].isnull().sum()!=var_dict[year].shape[0]))
        ].index
    ]
    impute_var_df_list = []
    test_scores = []
    for i, col in enumerate(var_dict[year].loc[0, 
                                               ((var_dict[year].isnull().sum()>0)&
                                                (var_dict[year].isnull().sum()!=var_dict[year].shape[0]))
                                              ].index):

        xgr_impute = XGBRegressor()
        df_temp = df.loc[df_target.iloc[:, i].notnull()]
        df_target_temp = df_target.iloc[:, i].dropna()

        X_train, X_test, y_train, y_test = train_test_split(df_temp, df_target_temp)
        xgr_impute.fit(X_train, y_train)
        print(col)
        test_scores.append(xgr_impute.score(X_test, y_test))

        impute_var_df_list.append(pd.DataFrame(xgr_impute.predict(df), columns=[col]))
    print(year, "average test score: ", np.mean(test_scores))
    df_impute = reduce(lambda left, right: pd.merge(
        left, right, left_index=True, right_index=True
    ), impute_var_df_list
                      )
    for i in range(df_target.shape[0]):
        for j in range(df_target.shape[1]):
            if df_target.iloc[i, j] != df_target.iloc[i, j]:
                df_target.iloc[i, j] = df_impute.iloc[i, j]
    for col in df_target.columns:
        var_dict[year].loc[:, col] = df_target.loc[:, col]

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[11:27:59] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
DP02_0037PE
[11:28:03] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


DP02_0044PE
[11:28:07] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

DP04_0114PE
[11:58:58] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


DP04_0115PE
2015 average test score:  0.4507937341725967
2016 average test score:  nan


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


TypeError: reduce() of empty sequence with no initial value

In [500]:
for year in range(2010, 2018):
    var_dict[year].dropna(axis=1, inplace=True)
    
    var_dict[year] = pd.concat([var_dict[year].FIPS,
                    var_dict[year].loc[:, var_dict[year].columns.str.contains(r'[0-9]PE')]
                    .loc[:, ((var_dict[year].loc[:, var_dict[year].columns.str.contains(r'[0-9]PE')].max()<100)&
                             (var_dict[year].loc[:, var_dict[year].columns.str.contains(r'[0-9]PE')].min()>=0))]],
                     axis=1
                   )

In [480]:
election = pd.read_csv("../data/countypres_2000-2016.csv")

In [481]:
election.dropna(inplace=True)

In [483]:
election["candidateshare"] = election.candidatevotes / election.totalvotes
hillary = election.loc[((election.year==2016)&(election.candidate=="Hillary Clinton"))]
hillary = hillary[["state", "county", "candidateshare", "FIPS"]]
hillary.rename(columns={"candidateshare":"hillary"}, inplace=True)
trump = election.loc[((election.year==2016)&(election.candidate=="Donald Trump"))]
trump = trump[["state", "county", "candidateshare", "FIPS"]]
trump.rename(columns={"candidateshare":"trump"}, inplace=True)

election_result_2016 = pd.merge(hillary, trump, on=["state", "county", "FIPS"])

election_result_2016["margin"] = (election_result_2016.trump - election_result_2016.hillary)
election_result_2016.FIPS = election_result_2016.FIPS.astype(int)
election_result_2016.head()

,state,county,hillary,FIPS,trump,margin
0,Alabama,Autauga,0.24,1001,0.73,0.49
1,Alabama,Baldwin,0.19,1003,0.77,0.57
2,Alabama,Barbour,0.47,1005,0.52,0.06
3,Alabama,Bibb,0.21,1007,0.76,0.55
4,Alabama,Blount,0.08,1009,0.89,0.81


In [484]:
obama = election.loc[((election.year==2012)&(election.candidate=="Barack Obama"))]
obama = obama[["state", "county", "candidateshare", "FIPS"]]
obama.rename(columns={"candidateshare":"obama"}, inplace=True)
romney = election.loc[((election.year==2012)&(election.candidate=="Mitt Romney"))]
romney = romney[["state", "county", "candidateshare", "FIPS"]]
romney.rename(columns={"candidateshare":"romney"}, inplace=True)

election_result_2012 = pd.merge(obama, romney, on=["state", "county", "FIPS"])

election_result_2012["margin"] = (election_result_2012.romney - election_result_2012.obama)
election_result_2012.FIPS = election_result_2012.FIPS.astype(int)
election_result_2012.head()

,state,county,obama,FIPS,romney,margin
0,Alabama,Autauga,0.27,1001,0.73,0.46
1,Alabama,Baldwin,0.22,1003,0.77,0.56
2,Alabama,Barbour,0.51,1005,0.48,-0.03
3,Alabama,Bibb,0.26,1007,0.73,0.47
4,Alabama,Blount,0.12,1009,0.86,0.74


In [501]:
var_dict_2016 = {}
for year in range(2010, 2018):
    var_dict_2016[year] = pd.merge(var_dict[year], 
              election_result_2016.drop(columns=["state", "county"]), 
              on="FIPS",
              how="inner").apply(pd.to_numeric, errors="ignore")

In [502]:
var_dict_2012 = {}
for year in range(2010, 2018):
    var_dict_2012[year] = pd.merge(var_dict[year], 
              election_result_2012.drop(columns=["state", "county"]), 
              on="FIPS",
              how="inner").apply(pd.to_numeric, errors="ignore")

In [506]:
var_dict_2016[2016].head()

,FIPS,DP02_0002PE,DP02_0003PE,DP02_0004PE,DP02_0005PE,DP02_0006PE,DP02_0007PE,DP02_0008PE,DP02_0009PE,DP02_0010PE,DP02_0011PE,DP02_0012PE,DP02_0013PE,DP02_0014PE,DP02_0018PE,DP02_0019PE,DP02_0020PE,DP02_0021PE,DP02_0022PE,DP02_0023PE,DP02_0025PE,DP02_0026PE,DP02_0027PE,DP02_0028PE,DP02_0029PE,DP02_0031PE,DP02_0032PE,DP02_0033PE,DP02_0034PE,DP02_0035PE,DP02_0053PE,DP02_0054PE,DP02_0056PE,DP02_0059PE,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE,DP02_0067PE,DP02_0069PE,DP02_0071PE,DP02_0073PE,DP02_0075PE,DP02_0077PE,DP02_0080PE,DP02_0081PE,DP02_0082PE,DP02_0083PE,DP02_0084PE,DP02_0085PE,DP02_0089PE,DP02_0090PE,DP02_0091PE,DP02_0092PE,DP02_0109PE,DP02_0112PE,DP02_0113PE,DP02_0114PE,DP02_0115PE,DP02_0116PE,DP02_0117PE,DP02_0118PE,DP02_0119PE,DP02_0120PE,DP02_0121PE,DP02_0123PE,DP02_0124PE,DP02_0125PE,DP02_0126PE,DP02_0127PE,DP02_0128PE,DP02_0129PE,DP02_0130PE,DP02_0131PE,DP02_0132PE,DP02_0133PE,DP02_0134PE,DP02_0135PE,DP02_0136PE,DP02_0137PE,DP02_0138PE,DP02_0139PE,DP02_0140PE,DP02_0141PE,DP02_0142PE,DP02_0143PE,DP02_0144PE,DP02_0145PE,DP02_0146PE,DP02_0147PE,DP02_0148PE,DP02_0149PE,DP03_0002PE,DP03_0003PE,DP03_0004PE,DP03_0005PE,DP03_0006PE,DP03_0007PE,DP03_0009PE,DP03_0011PE,DP03_0012PE,DP03_0013PE,DP03_0019PE,DP03_0020PE,DP03_0021PE,DP03_0022PE,DP03_0023PE,DP03_0024PE,DP03_0027PE,DP03_0028PE,DP03_0029PE,DP03_0030PE,DP03_0031PE,DP03_0033PE,DP03_0034PE,DP03_0035PE,DP03_0036PE,DP03_0037PE,DP03_0038PE,DP03_0039PE,DP03_0040PE,DP03_0041PE,DP03_0042PE,DP03_0043PE,DP03_0044PE,DP03_0045PE,DP03_0047PE,DP03_0048PE,DP03_0049PE,DP03_0050PE,DP03_0052PE,DP03_0053PE,DP03_0054PE,DP03_0055PE,DP03_0056PE,DP03_0057PE,DP03_0058PE,DP03_0059PE,DP03_0060PE,DP03_0061PE,DP03_0064PE,DP03_0066PE,DP03_0068PE,DP03_0070PE,DP03_0072PE,DP03_0074PE,DP03_0076PE,DP03_0077PE,DP03_0078PE,DP03_0079PE,DP03_0080PE,DP03_0081PE,DP03_0082PE,DP03_0083PE,DP03_0084PE,DP03_0085PE,DP03_0096PE,DP03_0097PE,DP03_0098PE,DP03_0099PE,DP03_0101PE,DP03_0105PE,DP03_0106PE,DP03_0107PE,DP03_0108PE,DP03_0116PE,DP03_0117PE,DP03_0119PE,DP03_0120PE,DP03_0122PE,DP03_0125PE,DP03_0128PE,DP03_0129PE,DP03_0130PE,DP03_0132PE,DP03_0133PE,DP03_0134PE,DP03_0135PE,DP03_0136PE,DP03_0137PE,DP04_0002PE,DP04_0003PE,DP04_0007PE,DP04_0008PE,DP04_0009PE,DP04_0010PE,DP04_0011PE,DP04_0012PE,DP04_0013PE,DP04_0014PE,DP04_0015PE,DP04_0017PE,DP04_0018PE,DP04_0019PE,DP04_0020PE,DP04_0021PE,DP04_0022PE,DP04_0023PE,DP04_0024PE,DP04_0025PE,DP04_0026PE,DP04_0028PE,DP04_0029PE,DP04_0030PE,DP04_0031PE,DP04_0032PE,DP04_0033PE,DP04_0034PE,DP04_0035PE,DP04_0036PE,DP04_0039PE,DP04_0040PE,DP04_0041PE,DP04_0042PE,DP04_0043PE,DP04_0044PE,DP04_0046PE,DP04_0047PE,DP04_0051PE,DP04_0052PE,DP04_0053PE,DP04_0054PE,DP04_0055PE,DP04_0056PE,DP04_0058PE,DP04_0059PE,DP04_0060PE,DP04_0061PE,DP04_0063PE,DP04_0064PE,DP04_0065PE,DP04_0066PE,DP04_0067PE,DP04_0068PE,DP04_0069PE,DP04_0070PE,DP04_0071PE,DP04_0073PE,DP04_0074PE,DP04_0075PE,DP04_0078PE,DP04_0079PE,DP04_0081PE,DP04_0082PE,DP04_0083PE,DP04_0084PE,DP04_0085PE,DP04_0086PE,DP04_0088PE,DP04_0091PE,DP04_0094PE,DP04_0096PE,DP04_0097PE,DP04_0098PE,DP04_0099PE,DP04_0100PE,DP04_0103PE,DP04_0104PE,DP04_0106PE,DP04_0107PE,DP04_0108PE,DP04_0112PE,DP04_0113PE,DP04_0114PE,DP04_0119PE,DP04_0120PE,DP04_0121PE,DP04_0122PE,DP04_0123PE,DP04_0124PE,DP04_0127PE,DP04_0128PE,DP04_0129PE,DP04_0130PE,DP04_0131PE,DP04_0132PE,DP04_0133PE,DP04_0138PE,DP04_0139PE,DP04_0140PE,DP04_0141PE,DP04_0142PE,DP05_0002PE,DP05_0003PE,DP05_0004PE,DP05_0005PE,DP05_0006PE,DP05_0007PE,DP05_0008PE,DP05_0009PE,DP05_0010PE,DP05_0011PE,DP05_0012PE,DP05_0013PE,DP05_0014PE,DP05_0015PE,DP05_0016PE,DP05_0018PE,DP05_0019PE,DP05_0020PE,DP05_0021PE,DP05_0023PE,DP05_0024PE,DP05_0026PE,DP05_0027PE,DP05_0030PE,DP05_0033PE,DP05_0034PE,DP05_0035PE,DP05_0036PE,DP05_0037PE,DP05_0038PE,DP05_0039PE,DP05_0040PE,DP05_0041PE,DP05_0042PE,DP05_0043PE,DP05_0044PE,DP05_0045PE,DP05_0046PE,DP05_0047PE,DP05_0048PE,DP05_0049PE,DP05_0050PE,DP05_0051PE,DP05_0052PE,DP05_0053PE,DP05_0054PE,DP05_0055PE,DP05_0056PE,DP05_0057P

In [509]:
X = var_dict_2016[2016].iloc[:, :-3]
y = var_dict_2016[2016].margin

In [510]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [511]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.iloc[:, 1:])
X_test_scaled = scaler.transform(X_test.iloc[:, 1:])

In [512]:
xgr = XGBRegressor()
xgr.fit(X_train_scaled, 
        y_train,
        eval_set=[(X_train_scaled, y_train), (X_test_scaled, y_test)],
        early_stopping_rounds=10,
        verbose=False)
print("train: ", xgr.score(X_train_scaled, y_train))
print("test: ", xgr.score(X_test_scaled, y_test))

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:28:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train:  0.9225834597284056
test:  0.8391941858507279


In [515]:
rfr = RandomForestRegressor()
rfr.fit(X_train_scaled, 
        y_train)
print("train: ", rfr.score(X_train_scaled, y_train))
print("test: ", rfr.score(X_test_scaled, y_test))

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


train:  0.9603786864744578
test:  0.7831190660844338


In [516]:
svr = SVR()
svr.fit(X_train_scaled, 
        y_train)
print("train: ", svr.score(X_train_scaled, y_train))
print("test: ", svr.score(X_test_scaled, y_test))

train:  0.9398598536583705
test:  0.8035007457859742


In [526]:
xgr = XGBRegressor(max_depth=4,
                   reg_alpha=5,
                   reg_lambda=30
                  )
xgr.fit(X_train_scaled, 
        y_train,
        eval_set=[(X_train_scaled, y_train), (X_test_scaled, y_test)],
        early_stopping_rounds=10,
        verbose=False)
print("train: ", xgr.score(X_train_scaled, y_train))
print("test: ", xgr.score(X_test_scaled, y_test))

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[12:35:01] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
train:  0.905660853753877
test:  0.8215430236167641


In [ ]:
xgr_scores = {}
xgr_feature_sets = []
k=0
for i in range(0, 10, 1):
    for j in range(0, 10, 1):
        X_train, X_test, y_train, y_test = train_test_split(X, y)
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train.iloc[:, 1:])
        X_test_scaled = scaler.transform(X_test.iloc[:, 1:])
        xgr = XGBRegressor(reg_alpha=i, reg_lambda=j)
        xgr.fit(X_train_scaled, y_train)

        
        xgr_features = pd.merge(pd.DataFrame(xgr.feature_importances_, 
                                             index=X.iloc[:, 1:].columns,
                                             columns=["importance"]
                                            ).sort_values(by="importance", 
                                                          ascending=False), 
                                var_description, 
                                how="left",
                                right_index=True, 
                                left_index=True)
        xgr_scores[k] = [i,
                         j,
                         xgr.score(X_train_scaled, y_train),
                         xgr.score(X_test_scaled, y_test),
                         np.sum(xgr_features.importance>0)]
        xgr_feature_sets.append(xgr_features)
        k += 1

/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:04:20] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:04:24] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


...

[13:04:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:04:43] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


/Users/kmiyaki/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[13:04:47] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
xgr_reg_score = pd.DataFrame(xgr_scores, 
                             index=["alpha",
                                    "lambda",
                                    "train",
                                    "test",
                                    "n_features"
                                   ]).T

In [ ]:
xgr_reg_score.sort_values(by="test", ascending=False).head(20)

In [ ]:
plt.scatter(xgr_reg_score.n_features, 
            xgr_reg_score.train,
            s=10,
            alpha=0.5
           )
plt.scatter(xgr_reg_score.n_features,
            xgr_reg_score.test,
            s=10,
            alpha=0.5
           )
plt.title("XGBoost Regressor: No of Features and Scores")
plt.ylabel("Score")
plt.xlabel("Number of Features")
plt.xlim(170, 0)
plt.legend(["Train","Test"])
plt.tight_layout()
plt.savefig("../images/xgr_reg_score")